# Movie Analysis: Obtaining Data 

## About:
In this notebook I will import the external data sets I gathered, turn them into tables to check on them and then save them as CSV files for further data scrubbing:

1. Money
2. Actors
3. Keywords

In [1]:
# imports for entire data gathering phase
import pandas as pd 
from bs4 import BeautifulSoup
import os

## 1. Money:
This dataset was scraped from the imdbpro website. It was behind authenticaton (really good authentication, owned by Amazon.com) and even though I have an account, I was unable to setup a normal scraping script.

I ended up logging into the site, and running a script in the developer console to scroll to the bottom (the content is lazy loaded, and scrolling down for an hour was not going to happen), and once at the bottom to save the page content.

In [2]:
def read_html(file_path):
    """Takes a saved html page and returns the results container that holds all the targeted info
    
    Arguments:
        file_path (string): path to find the saved html page

    Returns:
        result (BeautifulSoup object): or None if result wasn't found
    """
    # strip all the actor elements from the first html
    with open(file_path, 'r') as f_html:
        html = BeautifulSoup(f_html)
        # pull out only the elements we want
        result = html.find('div', id='results').ul.find_all('li', recursive=False)
        return result if result else None

In [3]:
# setup paths for importing raw html
movie_html_a_path = os.path.join(os.pardir, 'data', 'external', 'movie_money_1m_plus.htm')
movie_html_b_path = os.path.join(os.pardir, 'data', 'external', 'movie_money_1m_minus.htm')

In [4]:
money_a = read_html(movie_html_a_path)
money_b = read_html(movie_html_b_path)

In [5]:
# sanity check the results
print('type of container 1: {},\ntype of container 2: {}'.format(type(money_a), type(money_b)))
print('size of container 1: {},\nsize of container 2: {}'.format(len(money_a), len(money_b)))

type of container 1: <class 'bs4.element.ResultSet'>,
type of container 2: <class 'bs4.element.ResultSet'>
size of container 1: 7851,
size of container 2: 6851


In [6]:
# if they both look okay, go ahead and join them, then check the new length
money_a.extend(money_b)
len(money_a)

14702

In [21]:
# container parser
def money_parser(soup):
    """Parses through an array of soup objects and takes out the relevant info
    
    Arguments:
        soup (bs4.element.ResultSet): Chunk to search through
    
    Returns:
        results (List): A list of dictionaries 
    """
    results = []
    for title in soup:
        result = {
            'imdb_id': title.find('span', class_='display-title').a.get('href')[27:36] if title.find('span', class_='display-title') else None,
            'title': title.find('span', class_='display-title').a.get_text() if title.find('span', class_='display-title') else None,
            'year': title.find('span', class_='year').get_text()[1:-1] if title.find('span', class_='year') else None,
            'director': title.find('span', class_='display-name').a.get_text() if title.find('span', class_='display-name') else None,
            'production_co': title.find('span', class_='display-company').a.get_text() if title.find('span', class_='display-company') else None,
            'region_code': title.find('span', class_='region_code').get_text().strip() if title.find('span', class_='region_code') else None,
            'rank': title.find('span', class_='ranking').get_text().strip() if title.find('span', class_='ranking') else None,
            'budget_usd': title.find('span', class_='budget_usd').get_text().strip() if title.find('span', class_='budget_usd') else None,
            'us_gross': title.find('span', class_='us_gross').get_text().strip() if title.find('span', class_='us_gross') else None
        }
        results.append(result)
    return results

In [22]:
parsed_money = money_parser(money_a)

### Explore and make sure we have what we were looking for

In [23]:
# load up new dataset with the parsed info
money_test_df = pd.DataFrame.from_dict(parsed_money)
money_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14702 entries, 0 to 14701
Data columns (total 9 columns):
imdb_id          14700 non-null object
title            14700 non-null object
year             14685 non-null object
director         14671 non-null object
production_co    14345 non-null object
region_code      13447 non-null object
rank             14700 non-null object
budget_usd       14700 non-null object
us_gross         14700 non-null object
dtypes: object(9)
memory usage: 1.0+ MB


In [24]:
# look at the description
money_test_df.describe()

,imdb_id,title,year,director,production_co,region_code,rank,budget_usd,us_gross
count,14700,14700,14685,14671,14345,13447,14700,14700,14700
unique,14560,14204,105,6952,6232,90,14549,807,1370
top,tt6957966,Hamlet,2017,Woody Allen,Universal Pictures,[US],,,$1.1MM
freq,2,4,604,42,400,8886,12,6704,145


In [25]:
# look at some samples
money_test_df.sample(3)

,imdb_id,title,year,director,production_co,region_code,rank,budget_usd,us_gross
10086,tt0410400,On a Clear Day,2005,Gaby Dellal,Baker Street,[GB],"59,349",,$191K
14264,tt2520516,The Stroller Strategy,2012,Clément Michel,StudioCanal,[FR],"105,502",,$3K
11333,tt0269217,Tears of the Black Tiger,2000,Wisit Sasanatieng,Aichi Arts Center,[JP],"91,656",,$75K


In [26]:
# test for na values
money_test_df['us_gross'].isna().sum()

2

Look's like a pretty solid dataset. I will still have to format the currencies in the scrubbing phase. Let me save it to a CSV for the data scrubbing process:

In [27]:
# file out path. Going to put it in interim for now. After its scrubbed it will live in processed.
out_path = os.path.join(os.pardir, 'data', 'interim', 'money.csv')
money_test_df.to_csv(out_path, index=False)

In [38]:
money_df = pd.read_csv(out_path)
money_df['imdb_id'].sample(50)

5523     tt0334405
7618     tt0098660
3838     tt0139668
1497     tt0765446
2744     tt0098206
3636     tt0105415
6827     tt0082748
9446     tt0790706
8059     tt0087414
8328     tt3554418
13116    tt0143746
9560     tt0938341
5141     tt0093277
10091    tt0258854
1123     tt0082136
5650     tt0116684
7787     tt0238015
7408     tt0091083
6411     tt0096018
2169     tt0166485
2982     tt0489237
3307     tt4530422
2416     tt1306980
14298    tt0139898
4864     tt0098442
4949     tt0090073
6379     tt0118111
5738     tt0112368
2289     tt0171433
13811    tt0436857
2076     tt1320244
6126     tt2870708
11351    tt0148103
1351     tt0401855
1162     tt0113855
13137    tt3009772
4868     tt0306734
11953    tt2229842
9014     tt0803057
1551     tt0120694
8117     tt0346723
13540    tt0330247
1452     tt0316356
5675     tt0337876
8570     tt0091712
10936    tt2866360
10939    tt1220888
10753    tt0156096
5856     tt0081758
14268    tt2385952
Name: imdb_id, dtype: object

## 2. Actors:
Actors were also taken from the people section of the imdb pro website, in the same manner.

In [ ]:
# setup paths for importing raw html
actors_html_path = os.path.join(os.pardir, 'data', 'external', 'actors_0-10k.htm')
actors_html_b_path = os.path.join(os.pardir, 'data', 'external', 'actors_10k-20k.htm')
actors_html_c_path = os.path.join(os.pardir, 'data', 'external', 'actors_20k-30k.htm')

In [ ]:
# use our function to grab results div
actors_a = read_html(actors_html_path)
actors_b = read_html(actors_html_b_path)
actors_c = read_html(actors_html_c_path)

In [ ]:
# sanity check the results
print('type of container 1: {},\ntype of container 2: {},\ntype of container 3: {}'.format(type(actors_a), type(actors_b), type(actors_c)))
print('size of container 1: {},\nsize of container 2: {},\ntype of container 3: {}'.format(len(actors_a), len(actors_b), len(actors_c)))

Add them together:

In [ ]:
actors_a.extend(actors_b)
actors_a.extend(actors_c)

Check the size:

In [ ]:
len(actors_a)

In [ ]:
def parse_actors(soup):
    """Parses through an array of soup objects and takes out the relevant info
    
    Arguments:
        soup (bs4.element.ResultSet): Chunk to search through
    
    Returns:
        results (List): A list of dictionaries 
    """
    actors = []
    for actor_html in soup:
        actor = {
            'name': actor_html.find('span', class_='display-name').a.get_text() if actor_html.find('span', class_='display-name') else None,
            'year': actor_html.find('span', class_='year').get_text().strip() if actor_html.find('span', class_='year') else None,
            'rank': actor_html.find('span', class_='ranking').get_text().strip() if actor_html.find('span', class_='ranking') else None,
            'age': actor_html.find('span', class_='age_rank').get_text().strip() if actor_html.find('span', class_='age_rank') else None,
            'height': actor_html.find('span', class_='height').get_text().strip() if actor_html.find('span', class_='height') else None
        }
        actors.append(actor)
    return actors

In [ ]:
parsed_actors = parse_actors(actors_a)

In [ ]:
# load into df
actors_df = pd.DataFrame().from_dict(parsed_actors)

Checking out the result

In [ ]:
actors_df.info()

In [ ]:
actors_df.sample(5)

In [ ]:
actors_df.describe()

In [ ]:
actors_df.isna().sum()

Looks pretty good to me. Let's write it to a CSV and move on.

In [ ]:
# file out path. Going to put it in interim for now. After its scrubbed it will live in processed.
out_path = os.path.join(os.pardir, 'data', 'interim', 'actors.csv')
actors_df.to_csv(out_path, index=False)

In [ ]:
actors_df = pd.read_csv(out_path)
actors_df.head()

## 3. Keywords
The keywords data was gathered from the TheMovieDB.com site using the script in src/data/keyword_builder.

The script creates a list of imdb id's, and queries the API for each id. If it finds an id it will query the API once more to get the list of keywords for that id.

After the scripts runs through all the id's, it exports the database to a CSV and saves it in /data/raw.

## Finished
That is it for pulling in new data. We should have enough to answer our questions.